### IMPORTAR DATOS

In [8]:
import numpy as np
import pandas as pd
import os

### CARGAR DATOS

In [9]:

# Lista de posibles rutas
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/DGA.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/DGA.xlsx',
    'C:/Users/mticllacu/OneDrive - LUZ DEL SUR S.A.A/Archivos de Ronald Quispe Ocaña - ProyectoRyD_V2/Basededatos/DGA.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):   # verifica si existe
        df = pd.read_excel(path,header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/DGA.xlsx


In [10]:
df.head()

,Unnamed: 0,SERIE,FECHA DE MUESTRA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2,N2
0,NaN,146660T3,2022-05-12,179,53,17,314,139,1491,5095,10843,7977
1,NaN,146660T3,2017-03-10,75,42,29,134,10,962,1588,16321,19751
2,NaN,146660T3,2015-06-19,175,118,8,177,26,858,14735,15477,3838
3,NaN,146660T3,2013-10-22,163,264,17,21,30,537,15494,3637,17336
4,NaN,364076,2022-06-18,90,71,1,332,89,444,16063,19419,8659


### LIMPIEZA DE LOS DATOS

In [11]:
df = df.drop(columns=['Unnamed: 0'])
# df["TENSION"] = df["TENSION"].str.split("/").str[0]
df = df.iloc[:, :10]
if 'FECHA DE MUESTRA' in df.columns:
    df = df.rename(columns={'FECHA DE MUESTRA': 'FECHA'})
elif 'FECHA DE\nMUESTRA' in df.columns:
    df = df.rename(columns={'FECHA DE\nMUESTRA': 'FECHA'})
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
df = df.dropna(subset=['FECHA'])
df_full = df

In [12]:
df.head()

,SERIE,FECHA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2
0,146660T3,2022-05-12,179,53,17,314,139,1491,5095,10843
1,146660T3,2017-03-10,75,42,29,134,10,962,1588,16321
2,146660T3,2015-06-19,175,118,8,177,26,858,14735,15477
3,146660T3,2013-10-22,163,264,17,21,30,537,15494,3637
4,364076,2022-06-18,90,71,1,332,89,444,16063,19419


In [13]:
df_full = df.copy()
# ---------------------------
# LÍMITES Y GASES
# ---------------------------
limite = {
    'H2': {'concentracion': 150, 'tasa': 132},
    'CH4': {'concentracion': 130, 'tasa': 120},
    "C2H2": {"concentracion": 20, "tasa": 4},
    "C2H4": {"concentracion": 280, "tasa": 146},
    "C2H6": {"concentracion": 90, "tasa": 90},
    "CO": {"concentracion": 600, "tasa": 1060},
    "CO2": {"concentracion": 14000, "tasa": 10000}
}
gases2 = list(limite.keys())

In [14]:
# ---------------------------
# CONCENTRACIÓN
# ---------------------------
for gas in gases2:
    df[f'concentracion_{gas}'] = df.apply(lambda g: int(g[gas] > limite[gas]['concentracion']), axis=1)
df

,SERIE,FECHA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2,concentracion_H2,concentracion_CH4,concentracion_C2H2,concentracion_C2H4,concentracion_C2H6,concentracion_CO,concentracion_CO2
0,146660T3,2022-05-12,179,53,17,314,139,1491,5095,10843,1,0,0,1,1,1,0
1,146660T3,2017-03-10,75,42,29,134,10,962,1588,16321,0,0,1,0,0,1,0
2,146660T3,2015-06-19,175,118,8,177,26,858,14735,15477,1,0,0,0,0,1,1
3,146660T3,2013-10-22,163,264,17,21,30,537,15494,3637,1,1,0,0,0,0,1
4,364076,2022-06-18,90,71,1,332,89,444,16063,19419,0,0,0,1,0,0,1
5,364076,2017-06-11,51,29,13,312,103,755,5043,7267,0,0,0,1,1,1,0
6,364076,2016-01-15,57,73,14,196,74,717,11555,1349,0,0,0,0,0,1,0
7,364076,2014-11-15,81,154,25,170,63,1083,18431,9992,0,1,1,0,0,1,1
8,230531-01,2024-07-16,199,100,27,498,50,338,15162,19039,1,0,1,1,0,0,1
9,230531-01,2016-11-15,139,203,30,530,121,1222,14148,17669,0,1,1,1,1,1,1


In [15]:
# ---------------------------
# TASA DE INCREMENTO ANUAL
# ---------------------------
for gas in gases2:
    df[f"tasa_{gas}"] = df.groupby("SERIE").apply(
        lambda g: (g[gas].diff(-1) / ((g["FECHA"] - g["FECHA"].shift(-1)).dt.days) * 365)
    ).reset_index(level=0, drop=True)
df

C:\Users\roquispec\AppData\Local\Temp\ipykernel_51988\583415667.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[f"tasa_{gas}"] = df.groupby("SERIE").apply(
C:\Users\roquispec\AppData\Local\Temp\ipykernel_51988\583415667.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[f"tasa_{gas}"] = df.groupby("SERIE").apply(
C:\Users\roquispec\AppData\Local\Temp\ipykernel_51988\583415667.py:5: FutureWarning: DataFrame

,SERIE,FECHA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2,...,concentracion_C2H6,concentracion_CO,concentracion_CO2,tasa_H2,tasa_CH4,tasa_C2H2,tasa_C2H4,tasa_C2H6,tasa_CO,tasa_CO2
0,146660T3,2022-05-12,179,53,17,314,139,1491,5095,10843,...,1,1,0,20.095289,2.125463,-2.318687,34.780307,24.925887,102.215458,677.636316
1,146660T3,2017-03-10,75,42,29,134,10,962,1588,16321,...,0,1,0,-57.936508,-44.031746,12.166667,-24.912698,-9.269841,60.253968,-7616.912698
2,146660T3,2015-06-19,175,118,8,177,26,858,14735,15477,...,0,1,1,7.239669,-88.082645,-5.429752,94.115702,-2.413223,193.661157,-457.909091
3,146660T3,2013-10-22,163,264,17,21,30,537,15494,3637,...,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,364076,2022-06-18,90,71,1,332,89,444,16063,19419,...,0,0,1,7.765957,8.363339,-2.389525,3.982542,-2.787780,-61.928532,2194.380797
5,364076,2017-06-11,51,29,13,312,103,755,5043,7267,...,1,1,0,-4.269006,-31.306043,-0.711501,82.534113,20.633528,27.037037,-4633.294347
6,364076,2016-01-15,57,73,14,196,74,717,11555,1349,...,0,1,0,-20.563380,-69.401408,-9.424883,22.276995,9.424883,-313.591549,-5891.408451
7,364076,2014-11-15,81,154,25,170,63,1083,18431,9992,...,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,230531-01,2024-07-16,199,100,27,498,50,338,15162,19039,...,0,0,1,7.821429,-13.426786,-0.391071,-4.171429,-9.255357,-115.235714,132.182143
9,230531-01,2016-11-15,139,203,30,530,121,1222,14148,17669,...,1,1,1,28.123953,97.211055,5.502513,40.963149,1.834171,259.229481,4442.361809


In [ ]:
# ---------------------------
# FLAGS Y PUNTAJE
# ---------------------------
for gas, valores in limite.items():
    df[f"flag_{gas}"] = (df[f"tasa_{gas}"] > valores['tasa']).astype(int)

def puntaje_gas(row, gas):
    conc = row[f'concentracion_{gas}']
    tasa = row[f'flag_{gas}']
    if conc > 0 and tasa > 0:
        return 5
    elif conc > 0 and tasa <= 0:
        return 4
    elif conc <= 0 and tasa > 0:
        return 3
    else:
        return 1

for gas in gases2:
    df[f'puntaje_{gas}'] = df.apply(lambda row: puntaje_gas(row, gas), axis=1)
df

In [17]:
# ---------------------------
# CÁLCULO DGA
# ---------------------------
weight = {"C2H2":5,"H2":2,"CH4":3,"C2H4":4,"C2H6":3,"CO":2,"CO2":1}
total_weight = sum(weight.values())
df['DGA'] = df.apply(lambda row: sum(row[f'puntaje_{gas}']*weight[gas] for gas in gases2)/total_weight, axis=1)

df_DGA = df[['SERIE','FECHA','DGA']]
df_DGA

,SERIE,FECHA,DGA
0,146660T3,2022-05-12,2.65
1,146660T3,2017-03-10,2.30
2,146660T3,2015-06-19,1.75
3,146660T3,2013-10-22,1.90
4,364076,2022-06-18,1.75
5,364076,2017-06-11,2.35
6,364076,2016-01-15,1.30
7,364076,2014-11-15,2.65
8,230531-01,2024-07-16,2.80
9,230531-01,2016-11-15,3.95


### TRANSFORMACIÓN DE LOS DATOS

In [18]:

# ---------------------------
# EXTENSIÓN DEL CALENDARIO DESDE 2025
# ---------------------------
inicio = "2015-01-01"
desde_2025 = f"{pd.Timestamp.today().year}-01-01"
fecha_inicio = pd.Timestamp(inicio)  # en el 2026 cambiar ---****
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")

# Calendario
todas_series = df['SERIE'].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE","FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

# ---------- Tabla extendida de DGA ----------
ultimos_2024 = df_DGA[df_DGA['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024['FECHA'] = fecha_inicio
base_ext = pd.concat([df_DGA, ultimos_2024], ignore_index=True)

df_extendida = pd.merge(df_calendario, base_ext, on=["SERIE","FECHA"], how="left")
df_extendida = df_extendida.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# ---------- Tabla extendida de detalles ----------
ultimos_2024_det = df_full[df_full['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024_det['FECHA'] = fecha_inicio
base_ext_det = pd.concat([df_full, ultimos_2024_det], ignore_index=True)

df_extendida_detalles = pd.merge(df_calendario, base_ext_det, on=["SERIE","FECHA"], how="left")
df_extendida_detalles = df_extendida_detalles.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

C:\Users\roquispec\AppData\Local\Temp\ipykernel_51988\3178969255.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_extendida = df_extendida.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_51988\3178969255.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida = df_extendida.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_51988\3178969255

In [19]:
# ---------------------------
# DETALLES + DGA
# ---------------------------
df_detalles = pd.merge(df_full, df_DGA, on=["SERIE","FECHA"], how="left")
df_detalles_ext = pd.merge(df_extendida_detalles, df_extendida, on=["SERIE","FECHA"], how="left")

# Reordenar columnas: poner DGA después de FECHA
def reordenar(df_in):
    cols = list(df_in.columns)
    if "DGA" in cols:
        cols.remove("DGA")
        idx = cols.index("FECHA") + 1
        cols = cols[:idx] + ["DGA"] + cols[idx:]
    return df_in[cols]

df_detalles = reordenar(df_detalles)
df_detalles_ext = reordenar(df_detalles_ext)


In [20]:
# ---------------------------
# FUNCIONES PARA LLAMAR
# ---------------------------
def get_df_DGA():
    return df_DGA

def get_df_extendida_DGA():
    return df_extendida

def get_df_detalles_DGA():
    return df_detalles

def get_df_detalles_ext_DGA():
    return df_detalles_ext

print('\n ====== TABLA CON FECHAS ORIGINALES ====== \n')
print(get_df_DGA(), '\n')

print('\n ====== TABLA CON FECHAS EXTENDIDAS ====== \n')
print(get_df_extendida_DGA().head(), '\n')

print('\n ====== TABLA DE DETALLES DE DGA CON FECHAS ORIGINALES ====== \n')
print(get_df_detalles_DGA().head(), '\n')

print('\n ====== TABLA DE DETALLES DE DGA CON FECHAS EXTENDIDAS ====== \n')
print(get_df_detalles_ext_DGA().head(), '\n')


 ====== TABLA CON FECHAS ORIGINALES ====== 

        SERIE      FECHA   DGA
0    146660T3 2022-05-12  2.65
1    146660T3 2017-03-10  2.30
2    146660T3 2015-06-19  1.75
3    146660T3 2013-10-22  1.90
4      364076 2022-06-18  1.75
5      364076 2017-06-11  2.35
6      364076 2016-01-15  1.30
7      364076 2014-11-15  2.65
8   230531-01 2024-07-16  2.80
9   230531-01 2016-11-15  3.95
10  230531-01 2015-03-29  3.55
11  230531-01 2014-06-10  1.90
12     338118 2024-11-13  2.35
13     338118 2016-12-19  2.35
14     338118 2015-01-27  2.10
15     338118 2013-12-13  1.30 


 ====== TABLA CON FECHAS EXTENDIDAS ====== 

    SERIE      FECHA  DGA
0  338118 2015-01-01  1.3
1  338118 2015-01-02  1.3
2  338118 2015-01-03  1.3
3  338118 2015-01-04  1.3
4  338118 2015-01-05  1.3 


 ====== TABLA DE DETALLES DE DGA CON FECHAS ORIGINALES ====== 

      SERIE      FECHA   DGA   H2  CH4  C2H2  C2H4  C2H6    CO    CO2     O2
0  146660T3 2022-05-12  2.65  179   53    17   314   139  1491   5095  10843
1 

In [181]:
df.head(10)

,SERIE,TENSION,FECHA DE MUESTRA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,...,concentracion_C2H6,concentracion_CO,concentracion_CO2,tasa_H2,tasa_CH4,tasa_C2H2,tasa_C2H4,tasa_C2H6,tasa_CO,tasa_CO2
0,D518293,220,2025-02-07,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,...,0,1,0,0.691288,1.935606,0.0,0.0,0.414773,191.210227,323.522727
1,D518293,220,2023-08-29,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,...,0,1,0,1.655329,0.000000,0.0,0.0,0.000000,24.829932,158.911565
2,D518293,220,2022-06-14,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,...,0,1,0,31.063830,7.765957,0.0,0.0,0.000000,100.957447,613.510638
3,D518293,220,2022-04-28,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,...,0,1,0,8.554688,1.425781,0.0,0.0,1.425781,282.304688,808.417969
4,D518293,220,2021-08-15,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,...,0,0,0,2.619617,1.746411,0.0,0.0,0.000000,107.404306,792.870813
5,D518293,220,2020-06-23,10.0,5.0,1.0,1.0,1.0,424.0,1467.0,...,0,0,0,7.729412,0.858824,0.0,0.0,0.000000,64.411765,159.741176
6,D518293,220,2019-04-25,1.0,4.0,1.0,1.0,1.0,349.0,1281.0,...,0,0,0,0.000000,4.132075,0.0,0.0,0.000000,294.754717,1183.150943
7,D518293,220,2018-08-03,1.0,1.0,1.0,1.0,1.0,135.0,422.0,...,0,0,0,0.000000,0.000000,0.0,0.0,0.000000,127.260536,292.279693
8,D518293,220,2017-11-15,1.0,1.0,1.0,1.0,1.0,44.0,213.0,...,0,0,0,0.000000,0.000000,0.0,0.0,0.000000,451.904762,1998.809524
9,D518293,220,2017-10-25,1.0,1.0,1.0,1.0,1.0,18.0,98.0,...,0,0,0,0.000000,0.000000,0.0,0.0,0.000000,365.000000,1095.000000


In [13]:
# CREACIÓN DE NUEVA COLUMNA para definir los indices
for gas, valores in limite.items():
    col_tasa = f"tasa_{gas}"              # nombre de la columna en df
    col_flag = f"flag_{gas}"              # nueva columna binaria
    
    df[col_flag] = (df[col_tasa] > valores['tasa']).astype(int)

In [14]:
df.head(10)

,SERIE,TENSION,FECHA DE MUESTRA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,...,tasa_C2H6,tasa_CO,tasa_CO2,flag_H2,flag_CH4,flag_C2H2,flag_C2H4,flag_C2H6,flag_CO,flag_CO2
0,D518293,220,2025-02-07,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,...,0.414773,191.210227,323.522727,0,0,0,0,0,0,0
1,D518293,220,2023-08-29,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,...,0.000000,24.829932,158.911565,0,0,0,0,0,0,0
2,D518293,220,2022-06-14,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,...,0.000000,100.957447,613.510638,0,0,0,0,0,0,0
3,D518293,220,2022-04-28,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,...,1.425781,282.304688,808.417969,0,0,0,0,0,0,0
4,D518293,220,2021-08-15,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,...,0.000000,107.404306,792.870813,0,0,0,0,0,0,0
5,D518293,220,2020-06-23,10.0,5.0,1.0,1.0,1.0,424.0,1467.0,...,0.000000,64.411765,159.741176,0,0,0,0,0,0,0
6,D518293,220,2019-04-25,1.0,4.0,1.0,1.0,1.0,349.0,1281.0,...,0.000000,294.754717,1183.150943,0,0,0,0,0,0,0
7,D518293,220,2018-08-03,1.0,1.0,1.0,1.0,1.0,135.0,422.0,...,0.000000,127.260536,292.279693,0,0,0,0,0,0,0
8,D518293,220,2017-11-15,1.0,1.0,1.0,1.0,1.0,44.0,213.0,...,0.000000,451.904762,1998.809524,0,0,0,0,0,0,0
9,D518293,220,2017-10-25,1.0,1.0,1.0,1.0,1.0,18.0,98.0,...,0.000000,365.000000,1095.000000,0,0,0,0,0,0,0


In [15]:
# Función para asignar puntaje según concentración y tasa
def puntaje_gas(row, gas):
    conc = row[f'concentracion_{gas}']
    tasa = row[f'flag_{gas}']
    
    if conc > 0 and tasa > 0:  # ambas mayores que LMAX -> 5
        return 5
    elif conc > 0 and tasa <= 0:  # Concentración >, Tasa ≤ -> 4
        return 4
    elif conc <= 0 and tasa > 0:  # Concentración ≤, Tasa > -> 3
        return 3
    else:  # ambos ≤ -> 1
        return 1

# Aplicar para cada gas y crear columnas de puntaje
for gas in gases2:
    df[f'puntaje_{gas}'] = df.apply(lambda row: puntaje_gas(row, gas), axis=1)


In [185]:
df.head(20)

,SERIE,TENSION,FECHA DE MUESTRA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,...,flag_C2H6,flag_CO,flag_CO2,puntaje_H2,puntaje_CH4,puntaje_C2H2,puntaje_C2H4,puntaje_C2H6,puntaje_CO,puntaje_CO2
0,D518293,220,2025-02-07,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,...,0,0,0,1,1,1,1,1,4,1
1,D518293,220,2023-08-29,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,...,0,0,0,1,1,1,1,1,4,1
2,D518293,220,2022-06-14,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,...,0,0,0,1,1,1,1,1,4,1
3,D518293,220,2022-04-28,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,...,0,0,0,1,1,1,1,1,4,1
4,D518293,220,2021-08-15,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,...,0,0,0,1,1,1,1,1,1,1
5,D518293,220,2020-06-23,10.0,5.0,1.0,1.0,1.0,424.0,1467.0,...,0,0,0,1,1,1,1,1,1,1
6,D518293,220,2019-04-25,1.0,4.0,1.0,1.0,1.0,349.0,1281.0,...,0,0,0,1,1,1,1,1,1,1
7,D518293,220,2018-08-03,1.0,1.0,1.0,1.0,1.0,135.0,422.0,...,0,0,0,1,1,1,1,1,1,1
8,D518293,220,2017-11-15,1.0,1.0,1.0,1.0,1.0,44.0,213.0,...,0,0,0,1,1,1,1,1,1,1
9,D518293,220,2017-10-25,1.0,1.0,1.0,1.0,1.0,18.0,98.0,...,0,0,0,1,1,1,1,1,1,1


### Pesos

In [16]:
weight = {
    "C2H2": 5,
    "H2": 2,
    "CH4": 3,
    "C2H4": 4,
    "C2H6": 3,
    "CO": 2,
    "CO2": 1
}

In [17]:
total_weight = sum(weight.values())

# Cálculo del índice DGA
df['DGA'] = df.apply(
    lambda row: sum(row[f'puntaje_{gas}'] * weight[gas] for gas in gases2) / total_weight,
    axis=1
)

### Filtrar tabla

In [18]:
df_DGA = df[['SERIE', 'TENSION', 'FECHA DE MUESTRA', 'DGA']]

In [19]:
# Función para obtener DGA
def obtener_DGA(serie=None, fecha_muestra=None):
    resultado = df[['SERIE', 'TENSION', 'FECHA DE MUESTRA', 'DGA']]

    if serie is None and fecha_muestra is None:
        return resultado

    if serie is not None:
        resultado = resultado[resultado['SERIE'] == serie]

    if fecha_muestra is not None:
        fecha_muestra = pd.to_datetime(fecha_muestra)
        resultado = resultado[resultado['FECHA DE MUESTRA'] == fecha_muestra]

        if resultado.empty:
            return float('nan')
        return resultado['DGA'].values[0]

    return resultado

In [21]:
# PARA DGA
# 1. Toda la tabla
print(obtener_DGA())

# # 2. Solo por serie
# print(obtener_DGA('D518293'))

# # 3. Serie + fecha
# print(obtener_DGA('D518293', '2022-04-28'))


        SERIE TENSION FECHA DE MUESTRA  DGA
0     D518293     220       2025-02-07  1.3
1     D518293     220       2023-08-29  1.3
2     D518293     220       2022-06-14  1.3
3     D518293     220       2022-04-28  1.3
4     D518293     220       2021-08-15  1.0
...       ...     ...              ...  ...
2674   338118      60       2013-02-11  1.3
2675   338118      60       2011-12-19  1.3
2676   338118      60       2011-11-28  1.2
2677   338118      60       2011-11-21  1.0
2678   338118      60       2011-09-17  1.0

[2676 rows x 4 columns]


In [191]:
df_DGA

,SERIE,TENSION,FECHA DE MUESTRA,DGA
0,D518293,220,2025-02-07,1.3
1,D518293,220,2023-08-29,1.3
2,D518293,220,2022-06-14,1.3
3,D518293,220,2022-04-28,1.3
4,D518293,220,2021-08-15,1.0
...,...,...,...,...
2674,338118,60,2013-02-11,1.3
2675,338118,60,2011-12-19,1.3
2676,338118,60,2011-11-28,1.2
2677,338118,60,2011-11-21,1.0
